In [1]:
from dataclasses import field
from typing import ClassVar, Dict, Type

from ipyelk.diagram import layout_options as opt
from ipyelk.diagram.symbol import Def
from ipyelk.contrib.elements import Edge, Partition, Record, element
from ipyelk.contrib.shapes import connectors

In [2]:
from ipyelk.contrib.shapes.connectors import *

In [3]:
def CircleArrow(r=6, closed=False):
    return ConnectorDef(
        children=[
            Circle(x=r/4, y=0, radius=r/4),
            Path.from_list([(r, -r/2), (r/2, 0), (r, r/2)], closed=closed),
        ],
        correction=Point(-1, 0),
        offset=Point(-r / 2, 0),
    )

In [4]:
content_label_opts = opt.OptionsWidget(
    options=[opt.NodeLabelPlacement(horizontal="left", vertical="center")],
).value


top_center_label_opts = opt.OptionsWidget(
    options=[opt.NodeLabelPlacement(horizontal="center", vertical="top")],
).value


center_label_opts = opt.OptionsWidget(
    options=[opt.NodeLabelPlacement(horizontal="center", vertical="center")],
).value


bullet_opts = opt.OptionsWidget(
    options=[
        opt.LabelSpacing(spacing=4),
    ],
).value


compart_opts = opt.OptionsWidget(
    options=[
        opt.NodeSizeConstraints(),
    ],
).value


@element
class Block(Record):
    pass


@element
class Composition(Edge):
    shape_start: ClassVar[str] = "composition"


@element
class Aggregation(Edge):
    shape_start: ClassVar[str] = "aggregation"


@element
class Containment(Edge):
    shape_start: ClassVar[str] = "containment"


@element
class DirectedAssociation(Edge):
    shape_end: ClassVar[str] = "directed_association"


@element
class Association(Edge):
    pass


@element
class Generalization(Edge):
    shape_end: ClassVar[str] = "generalization"
        

@element
class Subsetting(Edge):
    shape_end: ClassVar[str] = "subsetting"


@element
class BlockDiagram(Partition):
    # TODO flesh out ideas of encapsulating diagram defs / styles / elements
    defs: ClassVar[Dict[str, Def]] = {
        "composition": connectors.Rhomb(r=4),
        "aggregation": connectors.Rhomb(r=4),
        "containment": connectors.Containment(r=4),
        "directed_association": connectors.StraightArrow(r=4),
        "generalization": connectors.StraightArrow(r=4, closed=True),
        "subsetting": CircleArrow(r=8, closed=False),
    }
    style: ClassVar[Dict[str, Def]] = {
        " .elklabel.compartment_title_1": {
            # "font-weight": "bold",
        },
        " .elklabel.heading, .elklabel.compartment_title_2": {
            "font-weight": "bold",
        },
        " .arrow.inheritance": {
            "fill": "none",
        },
        " .arrow.containment": {
            "fill": "none",
        },
        " .arrow.aggregation": {
            "fill": "none",
        },
        " .arrow.directed_association": {
            "fill": "none",
        },
        " .internal>.elknode": {
            "stroke": "transparent",
            "fill": "transparent",
        },
    }
    default_edge: Type[Edge] = field(default=Association)

In [5]:
import ipywidgets as W
from IPython.display import display

import ipyelk.nx
import ipyelk.tools
import ipyelk.tools.tools
from ipyelk import Elk
from ipyelk.contrib.elements import Compound, Edge, Label, Node, Port

# from ipyelk.contrib.elements.base import Element
from ipyelk.diagram import elk_model
from ipyelk.diagram import layout_options as opt
from ipyelk.diagram.symbol import ConnectorDef, Def, Symbol, symbols

In [6]:
import ipywidgets as W
import traitlets as T
from IPython.display import display

import ipyelk.nx
import ipyelk.tools
import ipyelk.tools.tools
from ipyelk import Elk
from ipyelk.contrib.elements import (
    Compartment,
    Compound,
    Edge,
    Label,
    Mark,
    Node,
    Port,
    Record,
)
# from ipyelk.contrib.library.block import Aggregation, Block, BlockDiagram, Composition, Generalization

# from ipyelk.contrib.elements.base import Element
from ipyelk.diagram import elk_model
from ipyelk.diagram import layout_options as opt
from ipyelk.diagram.symbol import ConnectorDef, Def, Symbol, symbols


class ToggleRecordBtn(ipyelk.tools.tools.ToggleCollapsedBtn):
    def get_related(self, node):
        tree = self.app.transformer.source[1]
        if isinstance(node, Mark) and isinstance(node.node, Compartment):
            parent = list(tree.predecessors(node))[0]
            return [child for i, child in enumerate(tree.neighbors(parent)) if i > 0]
        return super().get_related(node)


def block_app():
    """Utility function for creating a new Elk app suitable for an Activity Diagram"""
    diagram_opts = opt.OptionsWidget(
        options=[opt.Direction(value="RIGHT"), opt.HierarchyHandling()]
    ).value

    # configure app
    app = Elk(
        transformer=ipyelk.nx.XELK(
            layouts={
                elk_model.ElkRoot: {
                    "parents": diagram_opts,
                },
            },
        ),
        layout={"height": "100%"},
    )
    toggle = ToggleRecordBtn(app=app)
    fit = ipyelk.tools.tools.FitBtn(app=app)
    app.toolbar.commands = [fit, toggle]
    return app


def sysml2_parts_features():
    bd = BlockDiagram()
    
    item = Block(width=220)
    item.title = Compartment(headings=["«ItemDefinition»", "Item"])
    
    part = Block(width=220)
    part.title = Compartment(headings=["«PartDefinition»", "Part"])
    
    items = Block(width=220)
    items.title = Compartment(headings=["«ItemUsages»", "items"])
    items.properties = Compartment(content=["""multiplicity = "0..*" """])
    
    parts = Block(width=220)
    parts.title = Compartment(headings=["«PartUsages»", "parts"])
    parts.properties = Compartment(content=["""multiplicity = "0..*" """])
    
    action = Block(width=220)
    action.title = Compartment(headings=["«ActionDefinition»", "Action"])
    
    actions = Block(width=220)
    actions.title = Compartment(headings=["«ActionUsages»", "actions"])
    actions.properties = Compartment(content=["""multiplicity = "0..*" """])
    
    state_action = Block(width=220)
    state_action.title = Compartment(headings=["«StateDefinition»", "StateAction"])
    
    state_actions = Block(width=220)
    state_actions.title = Compartment(headings=["«StateUsages»", "stateActions"])
    state_actions.properties = Compartment(content=["""multiplicity = "0..*" """])
    
    bd[part:item:Generalization]
    bd[items:item:Generalization]
    bd[parts:items:Generalization]
    bd[parts:part:Generalization]
    bd[part:action:Subsetting]
    bd[action:actions:DirectedAssociation]
    bd[part:state_action:Subsetting]
    bd[state_action:state_actions:DirectedAssociation]
    
    # merge defs for both block and activities
    bd.defs = {**bd.defs}
    return bd


def example_sysml2():
    diagram = sysml2_parts_features()
    app = block_app()
    cp = Compound()
    app.transformer.source = cp(diagram)
    app.style = diagram.style
    app.diagram.defs = diagram.defs

    return app

In [7]:
BlockDiagram.style[' .elklabel.compartment_title_1'] = {}
BlockDiagram.style[' .elklabel.heading, .elklabel.compartment_title_2'] = {"font-weight": "bold"}

In [8]:
diagram_app = example_sysml2()
display(diagram_app)

Elk(children=[HTML(value='<style>.styled-widget-140075521962288 .elklabel.compartment_title_1{}.styled-widget-…

In [9]:
BlockDiagram.style

{' .elklabel.compartment_title_1': {},
 ' .elklabel.heading, .elklabel.compartment_title_2': {'font-weight': 'bold'},
 ' .arrow.inheritance': {'fill': 'none'},
 ' .arrow.containment': {'fill': 'none'},
 ' .arrow.aggregation': {'fill': 'none'},
 ' .arrow.directed_association': {'fill': 'none'},
 ' .internal>.elknode': {'stroke': 'transparent', 'fill': 'transparent'}}